In [ ]:
import numpy as np
import pandas as pd
import re # Used for the conversion of "r##_c##" in only the numbers --> TODO: check where it comes from
from IPython.display import display
from helpers import *
from play_with_data import *
from pre_processing import *
from matrix_factorization import *
from cross_validation import *
from apply_classifiers import *
from trainings_submissions import *
from regressions_models import *
import scipy.sparse as sp # In order to use sparse 
# Predictors imported in performance order (best to worst, according to http://surpriselib.com/)
from surprise import SVDpp
from surprise import KNNBaseline
from surprise import SVD
from surprise import SlopeOne
from surprise import BaselineOnly
from surprise import KNNWithMeans
from surprise import NMF
from surprise import CoClustering
from surprise import KNNBasic
from surprise import KNNWithZScore # not scored --> to be tested quickly
from surprise import dataset
from surprise import Dataset
from surprise import evaluate, print_perf
from surprise import GridSearch
from surprise import accuracy
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

%load_ext autoreload
%autoreload 2

In [ ]:
#******** Creation of a sparse matrix of the data **********
ratings = load_data('./data_train.csv')

In [ ]:
test = load_data('./sample_submission.csv')

# Hyperparameters optimization

In [ ]:
########### Define: algo, dataset (trainset ##############
ratings_hyper = formating_data_surprise(ratings)
#trainset_hyper = ratings_hyper.build_full_trainset()

In [ ]:
ratings_hyper.split(n_folds=7)

### Hyperparameters of SVDpp

In [ ]:
list_n_factors_pp = np.linspace(10,30,3)
list_lr_all_pp = np.logspace(-3.5,-2,3)
list_reg_all_pp = np.logspace(-3,-1.5,3)

In [ ]:
best_n_factors_pp = 0
best_lr_all_pp = 0
best_reg_all_pp = 0
best_rmse_pp = 1000

for n_factors in list_n_factors_pp:
    for lr_all in list_lr_all_pp:
        for reg_all in list_reg_all_pp:
            n_factors = int(n_factors)
            print(n_factors,lr_all,reg_all)

            algorithm_bsl = SVDpp(n_factors=n_factors,lr_all=lr_all,reg_all=reg_all)


            for trainset1, testset1 in ratings_hyper.folds():

                # train and test algorithm.
                algorithm_bsl.train(trainset1)
                predictions1 = algorithm_bsl.test(testset1)

                # Compute and print Root Mean Squared Error
                rmse = accuracy.rmse(predictions1, verbose=True)
                break
            if rmse < best_rmse_pp:
                print("Improvement!")
                best_n_factors_pp = n_factors
                best_lr_all_pp = lr_all
                best_reg_all_pp = reg_all
                best_rmse_pp = rmse


In [ ]:
print(best_n_factors_pp,best_lr_all_pp,best_reg_all_pp,best_rmse_pp)

10 0.00177827941004 0.001 0.985924781713

### Hyperparameters of NMF

In [ ]:
list_n_factors_nmf = np.linspace(10,20,3)
list_reg_pu = np.logspace(-2.5,-1,3)
list_reg_qi = np.logspace(-2.5,-1,3)

In [ ]:
best_n_factors_nmf = 0
best_reg_pu = 0
best_reg_qi = 0
best_rmse_nmf = 1000

for n_factors in list_n_factors_nmf:
    for reg_pu in list_reg_pu:
        for reg_qi in list_reg_qi:
            n_factors = int(n_factors)
            print(n_factors,reg_pu,reg_qi)

            algorithm_bsl = NMF(n_factors=n_factors,reg_pu=reg_pu,reg_qi=reg_qi)


            for trainset1, testset1 in ratings_hyper.folds():

                # train and test algorithm.
                algorithm_bsl.train(trainset1)
                predictions1 = algorithm_bsl.test(testset1)

                # Compute and print Root Mean Squared Error
                rmse = accuracy.rmse(predictions1, verbose=True)
                break
            if rmse < best_rmse_nmf:
                print("Improvement!")
                best_n_factors_nmf = n_factors
                best_reg_pu = reg_pu
                best_reg_qi = reg_qi
                best_rmse_nmf = rmse


In [ ]:
print(best_n_factors_nmf,best_reg_pu,best_reg_qi,best_rmse_nmf)

5 0.00177827941004 0.001
RMSE: 1.0029

# DON'T CARE

### Question

   - Il est normal de mettre un threshold (min_num_ratings) parce qu'on veut un certain nombre de données pour ce prononcer quand à donner une note, ceci implique qu'on enlève des users et des items, du coup notre matrice des ratings va être plus petite. Quand on la remplis et on fait un submit, comment est-ce qu'on gére ça?
       - Ici je suppose que c'est bien de faire la selection des ratings pour le train et test --> comme ceci on est pas ou moins biaisé par les movies et users qui n'ont que des 0 et qui donc n'apporte rien apart du "bruit"
       - Pour le remplissage de ce que l'on doit submit j'ai fait une petite comparaison entre les data que l'on nous donne et ce qui se trouve dans le sample_submission, (enfin je vais le faire) 

In [ ]:
#********** Pre-processing **********
min_num_ratings = 10 # or 15 this is based on the information given above
ratings_valid = valid_ratings(ratings, num_items_per_user, num_users_per_item,min_num_ratings)
train, test = split_data(ratings_valid) # This will put 90% of the items for the users that have at least one non-zero entry and the 10% in test


In [ ]:
cross_validation_application(train ,False ,4,0.01, 20, 0.2, 0.3,1)

In [ ]:
#********* Try ************
#rmse = rmse_movie_mean(train, test)
rmse = matrix_factorization_SGD(train, test)